#  Retrieval-Augmented Generation (RAG) with Shivaay API

This notebook demonstrates a simple Retrieval-Augmented Generation (RAG) pipeline using the **Shivaay LLM API**.

##  Goal:
Answer complex user questions more accurately by:
1. Generating **optimized search queries** from a user's question.
2. Retrieving relevant **contextual documents** (mocked here).
3. Asking the LLM to **answer using only that context**, reducing hallucinations.

##  Pipeline Steps:
1. **Generate Search Queries** – Get smarter queries to search for info.
2. **Retrieve Context** – (Mocked) In practice, pull data using a vector database or web search.
3. **Generate Answer with RAG** – Ask Shivaay to answer using only the given context.

>  Make sure to replace `"<YOUR-API-KEY>"` with your actual API key.



##  Step 1: Import Required Library & Configure Shivaay API

We start by importing `requests` to make API calls and configure our Shivaay API key and endpoint.


In [1]:
import requests

API_KEY = "<YOUR-API-KEY>"
API_URL = "https://api.futurixai.com/api/shivaay/v1/chat/completions"


##  Step 2: Generate Search Queries Using LLM

This function sends a user query to Shivaay and asks it to generate 3 optimized search queries that can be used to retrieve relevant information. These act as better search terms for information retrieval.


In [2]:
def generate_search_queries(user_query):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }

    payload = {
        "model": "shivaay",
        "messages": [
            {
                "role": "system",
                "content": "Generate 3 optimized search queries to find relevant information for answering the user's question."
            },
            {
                "role": "user",
                "content": user_query
            }
        ],
        "temperature": 0.3,
        "max_tokens": 150
    }

    response = requests.post(API_URL, headers=headers, json=payload)
    queries = response.json()['choices'][0]['message']['content'].split('\n')
    return [q.strip('- ').strip() for q in queries if q.strip()]


##  Step 3: Generate Final Answer Using Retrieved Context

This function sends both the user query and the retrieved document context to Shivaay, instructing it to answer based only on that context. If the context is not sufficient, it will respond with "I don't have enough information".


In [3]:
def answer_with_rag(user_query, context):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }

    payload = {
        "model": "shivaay",
        "messages": [
            {
                "role": "system",
                "content": f"""Answer the question using only the provided context. If unsure, say "I don't have enough information".

                Context:
                {context}"""
            },
            {
                "role": "user",
                "content": user_query
            }
        ],
        "temperature": 0.1,
        "max_tokens": 200
    }

    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()['choices'][0]['message']['content']


##  Step 4: Example Usage

We:
1. Provide a sample user question
2. Generate search queries
3. Define mock context (replace with actual retrieved text in real use)
4. Generate a final answer using that context

This demonstrates the RAG workflow from query to answer.


In [4]:
user_question = "What are the most promising battery technologies for grid storage?"
search_queries = generate_search_queries(user_question)
print("Generated search queries:", search_queries)

mock_context = """
1. Solid-state batteries show promise for grid storage with higher energy density.
2. Flow batteries are gaining traction for large-scale applications due to scalability.
3. Lithium-ion remains dominant but research continues into sodium-ion alternatives.
"""

final_answer = answer_with_rag(user_question, mock_context)
print("\nFinal Answer:", final_answer)


Generated search queries: ['1. "Most promising battery technologies for grid storage 2023"', '2. "Advancements in battery technology for large scale energy storage solutions"', '3. "Comparative analysis of emerging battery technologies for grid-scale applications"']

Final Answer: Based on the provided context, the most promising battery technologies for grid storage include solid-state batteries due to their higher energy density and flow batteries because of their scalability for large-scale applications. Additionally, lithium-ion batteries remain dominant, although there is ongoing research into sodium-ion alternatives as well.
